In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from matpower import start_instance

In [3]:
m = start_instance()

In [4]:
m.eval("warning('off', 'Octave:classdef-to-struct');")

In [5]:
mpc = m.loadcase("case9", verbose=False)
mpc

{'version': '2',
 'baseMVA': 100.0,
 'bus': array([[  1. ,   3. ,   0. ,   0. ,   0. ,   0. ,   1. ,   1. ,   0. ,
         345. ,   1. ,   1.1,   0.9],
        [  2. ,   2. ,   0. ,   0. ,   0. ,   0. ,   1. ,   1. ,   0. ,
         345. ,   1. ,   1.1,   0.9],
        [  3. ,   2. ,   0. ,   0. ,   0. ,   0. ,   1. ,   1. ,   0. ,
         345. ,   1. ,   1.1,   0.9],
        [  4. ,   1. ,   0. ,   0. ,   0. ,   0. ,   1. ,   1. ,   0. ,
         345. ,   1. ,   1.1,   0.9],
        [  5. ,   1. ,  90. ,  30. ,   0. ,   0. ,   1. ,   1. ,   0. ,
         345. ,   1. ,   1.1,   0.9],
        [  6. ,   1. ,   0. ,   0. ,   0. ,   0. ,   1. ,   1. ,   0. ,
         345. ,   1. ,   1.1,   0.9],
        [  7. ,   1. , 100. ,  35. ,   0. ,   0. ,   1. ,   1. ,   0. ,
         345. ,   1. ,   1.1,   0.9],
        [  8. ,   1. ,   0. ,   0. ,   0. ,   0. ,   1. ,   1. ,   0. ,
         345. ,   1. ,   1.1,   0.9],
        [  9. ,   1. , 125. ,  50. ,   0. ,   0. ,   1. ,   1. ,   0. ,
     

In [6]:
m.runpf(mpc, verbose=True)



MATPOWER Version 8.0, 17-May-2024
Power Flow -- AC-polar-power formulation

Newton's method converged in 4 iterations.
PF successful

Converged in 0.14 seconds
|     System Summary                                                           |

How many?                How much?              P (MW)            Q (MVAr)
---------------------    -------------------  -------------  -----------------
Buses              9     Total Gen Capacity     820.0        -900.0 to 900.0
Generators         3     On-line Capacity       820.0        -900.0 to 900.0
Committed Gens     3     Generation (actual)    319.6              22.8
Loads              3     Load                   315.0             115.0
  Fixed            3       Fixed                315.0             115.0
  Dispatchable     0       Dispatchable          -0.0 of -0.0      -0.0
Shunts             0     Shunt (inj)             -0.0               0.0
Branches           9     Losses (I^2 * Z)         4.64             48.38
Transformers   

{'version': '2',
 'baseMVA': 100.0,
 'bus': array([[  1.        ,   3.        ,   0.        ,   0.        ,
           0.        ,   0.        ,   1.        ,   1.04      ,
           0.        , 345.        ,   1.        ,   1.1       ,
           0.9       ],
        [  2.        ,   2.        ,   0.        ,   0.        ,
           0.        ,   0.        ,   1.        ,   1.025     ,
           9.28000548, 345.        ,   1.        ,   1.1       ,
           0.9       ],
        [  3.        ,   2.        ,   0.        ,   0.        ,
           0.        ,   0.        ,   1.        ,   1.025     ,
           4.66475133, 345.        ,   1.        ,   1.1       ,
           0.9       ],
        [  4.        ,   1.        ,   0.        ,   0.        ,
           0.        ,   0.        ,   1.        ,   1.02578839,
          -2.2167878 , 345.        ,   1.        ,   1.1       ,
           0.9       ],
        [  5.        ,   1.        ,  90.        ,  30.        ,
           0.   

In [7]:
m.push("_mpc", mpc, verbose=False)
m.eval("_r1 = runpf(_mpc);", verbose=True)
fieldnames_r1 = m.eval("fieldnames(_r1);", verbose=False)
fieldnames_r1.ravel()





MATPOWER Version 8.0, 17-May-2024
Power Flow -- AC-polar-power formulation

Newton's method converged in 4 iterations.
PF successful

Converged in 0.07 seconds
|     System Summary                                                           |

How many?                How much?              P (MW)            Q (MVAr)
---------------------    -------------------  -------------  -----------------
Buses              9     Total Gen Capacity     820.0        -900.0 to 900.0
Generators         3     On-line Capacity       820.0        -900.0 to 900.0
Committed Gens     3     Generation (actual)    319.6              22.8
Loads              3     Load                   315.0             115.0
  Fixed            3       Fixed                315.0             115.0
  Dispatchable     0       Dispatchable          -0.0 of -0.0      -0.0
Shunts             0     Shunt (inj)             -0.0               0.0
Branches           9     Losses (I^2 * Z)         4.64             48.38
Transformers 

Cell(['version', 'baseMVA', 'bus', 'gen', 'branch', 'gencost', 'order',
       'task', 'et', 'success', 'iterations', 'om'])

In [14]:
for i in fieldnames_r1.ravel():
    # print('start')
    # print(i)
    try:
        val = m.eval(f"_r1.{i};", verbose=True)
        # print(type(val))
        # print(val)
    except Exception as e:
        print(i)
        print(e)
        var_type = m.eval(f"class(_r1.{i});", verbose=False)
        print(var_type)
    # print('end')























    _pyeval>save_safe_struct at line 119 column 7
    _pyeval at line 97 column 3

    _pyeval>save_safe_struct at line 133 column 7
    _pyeval at line 97 column 3

task
Octave evaluation error:
error: save: error while writing '' to MAT file
error: called from:
    _pyeval>save_safe_struct at line 133, column 7


mp.task_pf_legacy










    _pyeval>save_safe_struct at line 119 column 7
    _pyeval at line 97 column 3

    _pyeval>save_safe_struct at line 133 column 7
    _pyeval at line 97 column 3

om
Octave evaluation error:
error: save: error while writing '' to MAT file
error: called from:
    _pyeval>save_safe_struct at line 133, column 7


mp.math_model_pf_acps


In [15]:
for i in ["om"]:
    fieldnames_om = m.eval(f"fieldnames(_r1.{i});", verbose=False)
    print(fieldnames_om.ravel())

['aux_data' 'element_classes' 'elements' 'lin' 'nlc' 'nle' 'nli'
 'prob_type' 'qdc' 'set_types' 'soln' 'userdata' 'var']


In [16]:
for i in fieldnames_om.ravel():
    # print('start')
    # print(i)
    try:
        val = m.eval(f"_r1.om.{i};", verbose=False)
        # print(type(val))
        # print(val)
    except Exception as e:
        print("start")
        print(i)
        print(e)
        var_type = m.eval(f"class(_r1.om.{i});", verbose=False)
        print(var_type)
        print("end")
        pass
    # print('end')

start
element_classes
Octave evaluation error:
error: save: error while writing '' to MAT file
error: called from:
    _pyeval>save_safe_struct at line 133, column 7
cell
end






start
elements
'NoneType' object is not iterable


mp.mapped_array
end
start
nle
Octave evaluation error:
error: save: error while writing '' to MAT file
error: called from:
    _pyeval>save_safe_struct at line 133, column 7
struct
end
